## Sandbox

Un  `.ipynb` pour tester des idées :)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));

In [ ]:
infos = open(
    readInstance,
    joinpath("data", "100_USA-road-d.BAY.gr"),
);
graph = graphFromData(infos);

In [ ]:
# foreach(((key, value),) -> println("$key = $value"), pairs(infos))

In [ ]:
# foreach(i -> println(graph[i]), labels(graph))

In [ ]:
# foreach(((i, j),) -> println(graph[i, j]), edge_labels(graph))

In [ ]:
function flow_creation(i::Int64)::Int64
    return Int64(graph[i].is_s) - Int64(graph[i].is_t)
end

In [ ]:
# __________MODELE_MAITRE__________
#declaration du modele
modele_maitre = Model(Gurobi.Optimizer)
modele_maitre = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(modele_maitre)
#declaration des variables
@variable(modele_maitre, z >= 0)
@variable(modele_maitre, a[edge_labels(graph)], Bin)  # vaut 1 ssi arc (i,j) choisi
@objective(modele_maitre, Min, z)
# declaration des contraintes
@constraint(modele_maitre, sum(graph[i, j].d * a[(i, j)] for (i, j) in edge_labels(graph)) <= z )
# Conservation du flot :
@constraint(
modele_maitre,
flow_cons[i in labels(graph)],
(
    sum(a[(i, j)] for j in outneighbor_labels(graph, i))
    -
    sum(a[(j, i)] for j in inneighbor_labels(graph, i))
    == flow_creation(i)
))
@constraint(modele_maitre, sum(graph[i].p * a[(i, j)] for (i, j) in edge_labels(graph))
+ graph[graph[].t].p <= graph[].big_s )
#__________________________________

In [ ]:
#__________SP1__________
#declaration du modele
SP1 = Model(Gurobi.Optimizer)
SP1 = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(SP1)
#declaration des variables
@variable(SP1, delta_d[edge_labels(graph)] >= 0)
#declaration des contraintes
@constraint(SP1, [(i, j) in edge_labels(graph)], delta_d[(i,j)] <= graph[i,j].big_d )
@constraint(SP1, sum(delta_d[(i,j)] for (i, j) in edge_labels(graph)) <= graph[].d1 )
#_______________________

#__________SP2__________
#declaration du modele
SP2 = Model(Gurobi.Optimizer)
SP2 = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(SP2)
#declaration des variables
@variable(SP2, delta_p[labels(graph)] >= 0)
#declaration des contraintes
@constraint(SP2, [i in labels(graph)], delta_p[i] <= 2 )
@constraint(SP2, sum(delta_p[i] for i in labels(graph)) <= graph[].d2 )
#_______________________

In [ ]:
function resolution_SP1(a_val, z_val)
    # Résolution SP1
    @objective(SP1, Max, sum(graph[i, j].d * (1 + delta_d[(i,j)]) * a_val[(i, j)] for (i, j) in edge_labels(graph)) )
    JuMP.optimize!(SP1)
    SP1_value = JuMP.objective_value(SP1)
    # println("Objective value SP1: ", SP1_value)
    delta_d_sol = JuMP.value.(delta_d)
    return SP1_value, delta_d_sol
end

function resolution_SP2(a_val, z_val)
    # Résolution SP2
    t = graph[].t
    @objective(SP2, Max,
    sum((graph[i].p + delta_p[i] * graph[i].ph) * a_val[(i, j)] for (i, j) in edge_labels(graph)) +
    graph[t].p + delta_p[t] * graph[t].ph )
    JuMP.optimize!(SP2)
    # temp = JuMP.value.(delta_d)
    # for (i, j) in edge_labels(graph)
    #     if temp[(i,j)] > 0
    #         println("(",i,",",j,") = ", temp[(i,j)])
    #     end
    # end
    SP2_value = JuMP.objective_value(SP2)
    # println("Objective value SP2: ", SP2_value)
    delta_p_sol = JuMP.value.(delta_p)
    return SP2_value, delta_p_sol
end

function my_lazy_callback(cb_data)
    # On récupère la valeur de a
    a_val = Dict((i,j) => callback_value(cb_data, a[(i,j)]) for (i,j) in edge_labels(graph))
    # a_val = callback_value.(cb_data, a)
    z_val = callback_value(cb_data, z)

    # Résolution SP1
    SP1_value, delta_d_sol = resolution_SP1(a_val, z_val)
    if (round(SP1_value - z_val; digits=4) > 0)
        cstr = @build_constraint(
            sum(graph[i, j].d * (1 + delta_d_sol[(i,j)]) * a[(i, j)] for (i, j) in edge_labels(graph)) <= z)
        MOI.submit(modele_maitre, MOI.LazyConstraint(cb_data), cstr)
        println("Added SP1 constraint")
        # println("Violée 1")
    end

    # Résolution SP2
    t = graph[].t
    SP2_value, delta_p_sol = resolution_SP2(a_val, z_val)
    if (SP2_value > graph[].big_s)
        cstr = @build_constraint(
            sum((graph[i].p + delta_p_sol[i] * graph[i].ph) * a[(i, j)] for (i, j) in edge_labels(graph)) +
            graph[t].p + delta_p_sol[t] * graph[t].ph <= graph[].big_s
        )
        MOI.submit(modele_maitre, MOI.LazyConstraint(cb_data), cstr)
        println("Added SP2 constraint")
        # println("Violée 2")
    end
end

In [ ]:
ResultWrapper = NamedTuple{
    (:primal_status, :dual_status, :term_status, :obj_value, :bound, :a),
    Tuple{
        ResultStatusCode,
        ResultStatusCode,
        TerminationStatusCode,
        Float64,
        Float64,
        Dict{Tuple{Int64, Int64}, Float64},
    }
}

function branch_and_cut(
    graph::MetaGraph;
    timelimit::Float64=3600.0
)::ResultWrapper
    start = time()
    s::Int64 = graph[].s
    t::Int64 = graph[].t
    # while (time() - start < timelimit)
    # Activation du lazycallback
    MOI.set(modele_maitre, MOI.LazyConstraintCallback(), my_lazy_callback)
    JuMP.optimize!(modele_maitre)
    m_value = JuMP.objective_value(modele_maitre)
    # end
    println("elapsed time : ", time() - start)
    return (
        primal_status=primal_status(modele_maitre),
        dual_status=dual_status(modele_maitre),
        term_status=termination_status(modele_maitre),
        obj_value=objective_value(modele_maitre),
        bound=objective_bound(modele_maitre),
        a=Dict{Tuple{Int64, Int64}, Float64}((i, j) => value(a[(i, j)]) for (i, j) in edge_labels(graph)),
    )
end


In [ ]:
function mkPath(a::Dict{Tuple{Int64, Int64}, Float64})::Vector{Int64}
    building_dict::Dict{Int64, Int64} = Dict{Int64, Int64}()
    for ((i, j), val) in pairs(a)
        int_val::Int64 = Int64(val)
        floor(val) == val && int_val in [0, 1] ? nothing : error("a is not integer feasible.")
        if int_val == 1
            haskey(building_dict, i) ? error("Two arcs coming from the same node.") : building_dict[i] = j
        end
    end
    current_i::Int64 = only(setdiff(keys(building_dict), values(building_dict)))
    path::Vector{Int64} = [current_i]
    while haskey(building_dict, current_i)
        current_i = building_dict[current_i]
        push!(path, current_i)
    end
    return path
end

In [ ]:
res = branch_and_cut(graph; timelimit=time() + 60.0)
println(mkPath(res.a))
println(res.obj_value)